In [ ]:
from numbalsoda import lsoda_sig, lsoda, rk45
from numba import njit, cfunc
from scipy.integrate import solve_ivp 
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
@cfunc(lsoda_sig)
def f(t, u, du, p):
    du[0] = u[0]-u[0]*u[1]
    du[1] = u[0]*u[1]-u[1]

@njit
def f_scipy(t, u):
    return np.array([u[0]-u[0]*u[1],u[0]*u[1]-u[1]])

In [ ]:
funcptr = f.address
u0 = np.array([5., 0.8])
data = np.array([1.0])
t0 = 0.0
tf = 50.0
itf = 1000
t_eval = np.linspace(t0, tf, itf)

In [ ]:
usol, success = lsoda(funcptr, u0, t_eval, data)

In [ ]:
plt.rcParams.update({'font.size': 15})
fig,ax = plt.subplots(1,1,figsize=[7,5])

ax.plot(t_eval,usol[:,0],label='u1')
ax.plot(t_eval,usol[:,1],label='u2')
ax.legend()
ax.set_xlabel('t')

plt.show()

In [ ]:
# Check that numbalsoda and scipy match for the hybrid LSODA solver.

# numbalsoda
usol, success = lsoda(funcptr, u0, t_eval, data, rtol = 1e-9, atol = 1e-30)

# scipy
t_span = [t0, tf]
sol = solve_ivp(f_scipy, t_span, u0, t_eval = t_eval,\
                rtol = 1e-9, atol = 1e-30, method='LSODA')
print(np.all(np.isclose(sol.y.T,usol)))

In [ ]:
# Check that numbalsoda and scipy match for the RK45 explicit solver.

# numbalsoda
tsol, usol, _, success = rk45(
    funcptr, u0, -1.0, t0, tf, 200, data, rtol = 1e-9, atol = 1e-30
)

# scipy
t_span = [t0, tf]
sol = solve_ivp(f_scipy, t_span, u0, t_eval=tsol,\
                rtol = 1e-9, atol = 1e-30, method='RK45')
print(np.all(np.isclose(sol.y.T,usol)))

In [ ]:
t_nb_lsoda = %timeit -o usol, success = lsoda(funcptr, u0, t_eval, data, rtol = 1e-6, atol = 1e-30)
t_sp = %timeit -o sol = solve_ivp(f_scipy, t_span, u0, t_eval = t_eval,\
                                  rtol = 1e-6, atol = 1e-30, method='LSODA')

print("\nscipy took "+'%i'%(t_sp.average/t_nb_lsoda.average)+" times longer than numbalsoda (LSODA integration)")

In [ ]:
t_nb_rk45 = %timeit -o tsol, usol, _, success = rk45(funcptr, u0, -1.0, t0, tf, 200, data, rtol = 1e-6, atol = 1e-30)
t_sp = %timeit -o sol = solve_ivp(f_scipy, t_span, u0, t_eval = tsol,\
                                  rtol = 1e-6, atol = 1e-30, method='RK45')

print("\nscipy took "+'%i'%(t_sp.average/t_nb_rk45.average)+" times longer than numbalsoda (RK45 integration)")

In [ ]:
# numbalsoda within jit compiled function works
@njit
def test():
    usol, success = lsoda(funcptr, u0, t_eval, data)
    return usol
test()

In [ ]:
# scipy within jit compiled function does not work
@njit
def test_sp():
    sol = solve_ivp(f_scipy, t_span, u0, t_eval = t_eval, method='LSODA')
    return sol
test_sp()